In [115]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Subset
from torch.utils.data import TensorDataset
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset,DataLoader,random_split

from torch import nn
import torch.nn.functional as F
import torch.optim as optim

## Load data

In [3]:
df_top5=pd.read_csv("top5.csv")
df_top10=pd.read_csv("top10.csv")
df_top100=pd.read_csv("top100.csv")

In [4]:
df_top10.head()

,Unnamed: 0,consid,lsquotelephon,unmatch,swintus,tokugawa,theyv,seamless,neighbear,obvious,...,tuareg,dicewithoutdic,ldquonordquo,resourcepoor,theft,parenthet,rogu,grunspan,piersmdashth,Description
0,148494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,the animals on the farm are playing a game of ...
1,316377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7 wonders the board game with more awards than...
2,155987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,the abyss power is once again vacant so the ti...
3,21569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,adigma is a party game which involves simultan...
4,31260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,description from boardgamenews in agricola you...


In [5]:
df_top5.head()

,Unnamed: 0,swintus,persuas,unmatch,tokugawa,suburbia,seamless,plastic,state,360,...,supermarket,taboo,export,pepper,concordia,maus,rogu,natur,orbit,Description
0,148494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,the animals on the farm are playing a game of ...
1,316377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7 wonders the board game with more awards than...
2,155987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,the abyss power is once again vacant so the ti...
3,21569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,adigma is a party game which involves simultan...
4,31260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,description from boardgamenews in agricola you...


In [6]:
df_top100.head()

,Unnamed: 0,player,game,card,one,play,point,new,take,use,...,combin,even,possibl,help,expans,money,locat,around,give,Description
0,148494,3,1,0,1,1,1,0,2,0,...,0,0,0,0,0,0,0,0,1,the animals on the farm are playing a game of ...
1,316377,4,9,8,1,3,2,1,0,1,...,0,0,0,0,0,0,0,0,0,7 wonders the board game with more awards than...
2,155987,4,2,3,1,0,0,0,2,3,...,1,0,0,0,0,0,2,0,0,the abyss power is once again vacant so the ti...
3,21569,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,adigma is a party game which involves simultan...
4,31260,4,5,7,3,2,0,1,2,3,...,0,0,1,0,0,0,0,0,0,description from boardgamenews in agricola you...


In [64]:
df= pd.DataFrame()

In [103]:
for column in df_top100.columns[:-1]:
    df[column]= df_top100[column].astype('float32')

### To tensor

In [104]:
target = torch.tensor(df.iloc[:,0].values)
features = torch.tensor(df.iloc[:,1:].values)
train = TensorDataset(features, target)

In [105]:
type(train)

torch.utils.data.dataset.TensorDataset

### Split data and dataloader

In [106]:
m=len(train)

In [107]:
train_data, test_data = random_split(train, [int(m-m*0.2), int(m*0.2)+1])

In [108]:
batch_size=25

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,shuffle=True)

In [109]:
for text, _ in train_loader:
    print(text.dtype)

torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32


In [83]:
train_features, train_labels = next(iter(train_loader))
print(train_features)
print(len(train_features))
print(train_features.shape)
print(train_labels)
print(train_features.dtype)

tensor([[7., 1., 1., 0., 2., 3., 0., 1., 0., 2., 0., 4., 3., 1., 0., 1., 0., 1.,
         0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 1., 0., 0., 0., 4., 0., 0., 2., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 2., 1., 0., 0., 1., 1., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 1., 0., 0., 0.]], dtype=torch.float64)
1
torch.Size([1, 100])
tensor([245658.], dtype=torch.float64)
torch.float64


In [84]:
X = torch.rand(1, 28, 28, device=device)

In [85]:
X.shape

torch.Size([1, 28, 28])

## Create Autoencoder

### NN class

In [110]:
class Encoder(nn.Module):
    
    def __init__(self, encoded_space_dim,fc2_input_dim):
        super().__init__()
        
        
        ### Flatten layer
        #self.flatten = nn.Flatten(start_dim=1)
        
### Linear section
        self.encoder_lin = nn.Sequential(
            nn.Linear(fc2_input_dim, 128),
            nn.ReLU(True),
            nn.Linear(128,64),
            nn.ReLU(True),
            nn.Linear(64, encoded_space_dim)
        )
        
    def forward(self, x):

        #x = self.flatten(x)
        x = self.encoder_lin(x)
        return x
class Decoder(nn.Module):
    
    def __init__(self, encoded_space_dim,fc2_input_dim):
        super().__init__()
        self.decoder_lin = nn.Sequential(
            nn.Linear(encoded_space_dim, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True),
            nn.Linear(128,fc2_input_dim)

        )

        
    def forward(self, x):
        x = self.decoder_lin(x)
        #x = self.unflatten(x)
        #x = self.decoder_conv(x)
        x = torch.sigmoid(x)
        return x

### Initialize

In [111]:
## Define the loss function
loss_fn = torch.nn.MSELoss()

### Define an optimizer (both for the encoder and the decoder!)
lr= 0.001

### Set the random seed for reproducible results
torch.manual_seed(0)

### Initialize the two networks
d = 4

#model = Autoencoder(encoded_space_dim=encoded_space_dim)
encoder = Encoder(encoded_space_dim=d,fc2_input_dim=100)
decoder = Decoder(encoded_space_dim=d,fc2_input_dim=100)
params_to_optimize = [
    {'params': encoder.parameters()},
    {'params': decoder.parameters()}
]

optim = torch.optim.Adam(params_to_optimize, lr=lr, weight_decay=1e-05)

# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

# Move both the encoder and the decoder to the selected device
print(encoder.to(device))
print(decoder.to(device))

Selected device: cpu
Encoder(
  (encoder_lin): Sequential(
    (0): Linear(in_features=100, out_features=128, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=4, bias=True)
  )
)
Decoder(
  (decoder_lin): Sequential(
    (0): Linear(in_features=4, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=128, out_features=100, bias=True)
  )
)


### Functions

#### Train

In [112]:
def train_epoch(encoder, decoder, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder
    encoder.train()
    decoder.train()
    train_loss = []
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for text, _ in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        # Move tensor to the proper device
        text = text.to(device)
        # Encode data
        encoded_data = encoder(text)
        # Decode data
        decoded_data = decoder(encoded_data)
        # Evaluate loss
        loss = loss_fn(decoded_data, text)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print batch loss
        #print('\t partial train loss (single batch): %f' % (loss.data))
        train_loss.append(loss.detach().cpu().numpy())

    return np.mean(train_loss)

#### Test

In [113]:
def test_epoch(encoder, decoder, device, dataloader, loss_fn):
    # Set evaluation mode for encoder and decoder
    encoder.eval()
    decoder.eval()
    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        for text, _ in dataloader:
            # Move tensor to the proper device
            text = text.to(device)
            # Encode data
            encoded_data = encoder(text)
            # Decode data
            decoded_data = decoder(encoded_data)
            # Append the network output and the original image to the lists
            conc_out.append(decoded_data.cpu())
            conc_label.append(text.cpu())
        # Create a single tensor with all the values in the lists
        conc_out = torch.cat(conc_out)
        conc_label = torch.cat(conc_label) 
        # Evaluate global loss
        val_loss = loss_fn(conc_out, conc_label)
    return val_loss.data

In [116]:
num_epochs = 30
diz_loss = {'train_loss':[],'val_loss':[]}
for epoch in range(num_epochs):
    train_loss =train_epoch(encoder,decoder,device,
    train_loader,loss_fn,optim)
    val_loss = test_epoch(encoder,decoder,device,test_loader,loss_fn)
    print('\n EPOCH {}/{} \t train loss {} \t val loss {}'.format(epoch + 1, num_epochs,train_loss,val_loss))
    diz_loss['train_loss'].append(train_loss)
    diz_loss['val_loss'].append(val_loss)

AttributeError: module 'torch.optim' has no attribute 'zero_grad'